Для работы с большим файлом, установим библиотеку

In [1]:
!pip install polars


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: nomeroff-net 3.1.1 has a non-standard dependency specifier numpy>=1.16.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nomeroff-net or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: nomeroff-net 3.1.1 has a non-standard dependency specifier imgaug>=0.4.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nomeroff-net or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: nomeroff-net 3.1.1 has a non-standard dependency specifier numpy>=1.16.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nomeroff-net or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: nomeroff-net 3.1.1 has a non-standard dependency specifier imgaug>=0.4.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of nomeroff-net or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Импортируем необходимые библиотеки.

In [3]:
import polars as pl
import plotly.express as px

pl.Config(tbl_cols=-1, tbl_rows=10)

GOST engine already loaded


Добавим данные в объекты Polaris

In [4]:
train_data = pl.scan_csv('data/train.csv')
lectures_data = pl.scan_csv('data/lectures.csv')
questions_data = pl.scan_csv('data/questions.csv')

Посмотрим первые 5 строк каждого файла для оценки данных

In [5]:
train_head = train_data.tail(5).collect()
lectures_head = lectures_data.head(5).collect()
questions_head = questions_data.head(5).collect()

print("TRAIN", train_head, "Lectures", lectures_head, "Questions", questions_head, sep="\n")



TRAIN
shape: (5, 10)
┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬────────┐
│ row_id  ┆ timesta ┆ user_id ┆ content ┆ content ┆ task_co ┆ user_an ┆ answere ┆ prior_q ┆ prior_ │
│ ---     ┆ mp      ┆ ---     ┆ _id     ┆ _type_i ┆ ntainer ┆ swer    ┆ d_corre ┆ uestion ┆ questi │
│ i64     ┆ ---     ┆ i64     ┆ ---     ┆ d       ┆ _id     ┆ ---     ┆ ctly    ┆ _elapse ┆ on_had │
│         ┆ i64     ┆         ┆ i64     ┆ ---     ┆ ---     ┆ i64     ┆ ---     ┆ d_time  ┆ _expla │
│         ┆         ┆         ┆         ┆ i64     ┆ i64     ┆         ┆ i64     ┆ ---     ┆ nation │
│         ┆         ┆         ┆         ┆         ┆         ┆         ┆         ┆ f64     ┆ ---    │
│         ┆         ┆         ┆         ┆         ┆         ┆         ┆         ┆         ┆ bool   │
╞═════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╪═════════╪════════╡
│ 1012303 ┆ 4285644 ┆ 2147482 ┆ 3586    ┆ 0       ┆ 22      ┆ 0       

Посмотрим количество уникальных студентов, что бы оценить объем выборки

In [6]:
train_data.select(pl.col('user_id').alias('Пользователи')).unique().count().collect(streaming=True)

Пользователи
u32
393656


Посмотрим сколько в базе всего вопросов, и сколько уникальных вопросов в выборке со студентами. Сравним одно с другим

In [7]:
qt = questions_data.select('question_id').count().collect()['question_id'][0]
tt = train_data.select('content_id').unique().count().collect()['content_id'][0]

print(f"Таблица вопросов {qt} \n Таблица ответов {tt} \n Сравним: {tt == qt}")

Таблица вопросов 13523 
 Таблица ответов 13782 
 Сравним: False


Отличается. Сделаем выборку проходящих тестирование

In [74]:
train_test = train_data.filter(
        pl.col('content_type_id') == 0
    )

Повторим проверку

In [9]:
qt = questions_data.select('question_id').count().collect()['question_id'][0]
tt = train_test.select('content_id').unique().count().collect()['content_id'][0]

print(f"Таблица вопросов {qt} \n Таблица ответов {tt} \n Сравним: {tt == qt}")

Таблица вопросов 13523 
 Таблица ответов 13523 
 Сравним: True


Одинаково. Теперь попробуем вывести топ 10 студентов по отвеченным вопросам

In [77]:
print (
    train_test
    .filter(pl.col("answered_correctly") == 1)  # Фильтрация
    .group_by("user_id")  # Группировка
    .agg([
        pl.len().alias("answered"),
        pl.col("content_id").n_unique().alias("unique_content_count")
    ])
    .sort("answered", descending=True)  # Сортировка
    .limit(10)  # Ограничение
    .collect(streaming=True) # Материализация LazyFrame в DataFrame
)

shape: (10, 3)
┌────────────┬──────────┬──────────────────────┐
│ user_id    ┆ answered ┆ unique_content_count │
│ ---        ┆ ---      ┆ ---                  │
│ i64        ┆ u32      ┆ u32                  │
╞════════════╪══════════╪══════════════════════╡
│ 2139561972 ┆ 14300    ┆ 9227                 │
│ 1615528747 ┆ 13678    ┆ 9245                 │
│ 338684437  ┆ 13416    ┆ 8153                 │
│ 1842816145 ┆ 13086    ┆ 7797                 │
│ 1047202059 ┆ 13034    ┆ 7158                 │
│ 455973631  ┆ 12840    ┆ 6360                 │
│ 1478712595 ┆ 12840    ┆ 7689                 │
│ 801103753  ┆ 12698    ┆ 7827                 │
│ 1743444187 ┆ 12644    ┆ 8081                 │
│ 2070068957 ┆ 12225    ┆ 9655                 │
└────────────┴──────────┴──────────────────────┘


Замечаем, что количество ответов больше чем уникальных вопросов.
Видно, что в действительности правильных уникальный ответов меньше в 1,54 раза, что явно смазывает картину успеваемости. 

Пытаемся выяснить в чём причина. Смотрим отдельно по студенту 2139561972.
Считаем количество правильных ответов и группируем по id вопроса

In [75]:
print(    
    train_test.filter((pl.col("answered_correctly") == 1) & (pl.col("user_id") == 2139561972))  # Фильтрация
    .group_by("content_id")  # Группировка по content_id
    .agg(pl.len().alias("counts"))  # Подсчёт количества строк
    .sort("counts", descending=True)  # Сортировка по counts
    .limit(10)  # Ограничение результата до первых 10 записей
    .collect()
)

shape: (10, 2)
┌────────────┬────────┐
│ content_id ┆ counts │
│ ---        ┆ ---    │
│ i64        ┆ u32    │
╞════════════╪════════╡
│ 4979       ┆ 5      │
│ 7543       ┆ 4      │
│ 6976       ┆ 4      │
│ 7485       ┆ 4      │
│ 7497       ┆ 4      │
│ 7342       ┆ 4      │
│ 7217       ┆ 4      │
│ 7345       ┆ 4      │
│ 6848       ┆ 4      │
│ 8018       ┆ 4      │
└────────────┴────────┘


Видим, что студент неоднократно проходил один и тот же вопрос. Подчистим выборку оставив только последние ответы студентов и сохраним новую выборку в parquet файл, для ускорения дальнейшей обработки.

In [29]:
unique_df = train_data.sort("row_id", descending=True).unique(subset=["user_id", "content_id"], keep="first")
unique_df.collect(streaming=True).write_parquet('data/train_uniquie.parquet')

Откроем новый файл с уникальными значения для работы с выборкой

In [30]:
train = pl.scan_parquet('data/train_uniquie.parquet')

In [32]:
train_test = train.filter(
        pl.col('content_type_id') == 0
    )

Проверим количество ответов и их уникальное количество

In [51]:
print (
    train_test
    .filter(pl.col("answered_correctly") == 1)  # Фильтрация
    .group_by("user_id")  # Группировка
    .agg([
        pl.len().alias("answered"),
        pl.col("content_id").n_unique().alias("unique_content_count")
    ])
    .sort("answered", descending=True)  # Сортировка
    .limit(10)  # Ограничение
    .collect(streaming=True) # Материализация LazyFrame в DataFrame
)


shape: (10, 3)
┌────────────┬──────────┬──────────────────────┐
│ user_id    ┆ answered ┆ unique_content_count │
│ ---        ┆ ---      ┆ ---                  │
│ i64        ┆ u32      ┆ u32                  │
╞════════════╪══════════╪══════════════════════╡
│ 1223856469 ┆ 10003    ┆ 10003                │
│ 2070068957 ┆ 9527     ┆ 9527                 │
│ 721126014  ┆ 9381     ┆ 9381                 │
│ 244146869  ┆ 9353     ┆ 9353                 │
│ 1509029498 ┆ 9231     ┆ 9231                 │
│ 1098525251 ┆ 9214     ┆ 9214                 │
│ 716820873  ┆ 9142     ┆ 9142                 │
│ 1615528747 ┆ 9126     ┆ 9126                 │
│ 2139561972 ┆ 9037     ┆ 9037                 │
│ 2132723519 ┆ 8998     ┆ 8998                 │
└────────────┴──────────┴──────────────────────┘


Теперь выглядит всё нормально. Попробуем дальше анализировать эту выборку.

Теперь посмотрим описание числовых показателей. Bool приведем к int8

In [17]:
train_stat = (train_test.select(
    # выбираем все столбцы, кроме перечисленных
    pl.all().exclude('row_id', 'content_type_id', 'user_id', 'content_id', 'user_answer', 'task_container_id', 'prior_question_had_explanation'),
    # столбец с bool значением приведем к числовому int8
    pl.col('prior_question_had_explanation').cast(pl.Int8)
).collect(streaming=True))
# вызовем методом вывода описания
train_stat.describe()

statistic,timestamp,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
str,f64,f64,f64,f64
"""count""",8.6809085e7,8.6809085e7,8.6416579e7,8.6416579e7
"""null_count""",0.0,0.0,392506.0,392506.0
"""mean""",7.2684e9,0.644929,25898.415826,0.896621
"""std""",1.1260e10,0.478535,20052.128906,0.304453
"""min""",0.0,0.0,0.0,0.0
"""25%""",4.33946245e8,0.0,16000.0,1.0
"""50%""",2.4271e9,1.0,21000.0,1.0
"""75%""",9.0322e9,1.0,30000.0,1.0
"""max""",8.7426e10,1.0,300000.0,1.0


При первоначальной оценке датасета видим, что процент верных ответов 69.4, при этом с объяснением предыдущего вопроса ознакомилось 90% учащихся. Так же видим большой временной разброс временных показателей.


Посмотрим зависимость правильных ответов от получения разъяснений на предыдущий вопрос.

In [40]:
train_test.select(
        pl.col('prior_question_had_explanation'),
        pl.col('answered_correctly'),
    ).drop_nulls().group_by('prior_question_had_explanation').agg(
        pl.col('answered_correctly')
        .mean()
    ).collect(streaming=True)



prior_question_had_explanation,answered_correctly
bool,f64
false,0.507902
true,0.714852


Учащийся отвечал на следующий вопрос в 71.5% верно, если до этого получал разъяснения, иначе только в 50.8%.

Посмотрим с другой стороны - как часто учащиеся получали разъяснения после правильного/неправильного ответа. Здесь нам придется сдвинуть строки в столбцах, чтобы разъяснение относилось к анализируемому вопросу

In [58]:
print(
    train_test.select(
        # возьмем столбец с разъяснениями
        pl.col('prior_question_had_explanation'),
        # возьмем столбец с ответами
        pl.col('answered_correctly')
    # сгруппируем данные по столбцу ответа
    ).group_by('answered_correctly')
    # для стобца факта разъяснений проведем аггрегацию
    .agg(
        pl.col('prior_question_had_explanation')
        # приводим bool к числу
        .cast(pl.Int8)
        # вычисляем среднее
        .mean()
        # так как смысл столбца поменялся, переименуем его
        .alias('question_explanation')
    # запускаем обработку со стримингом
    ).collect(streaming=True)
)

shape: (2, 2)
┌────────────────────┬──────────────────────┐
│ answered_correctly ┆ question_explanation │
│ ---                ┆ ---                  │
│ i64                ┆ f64                  │
╞════════════════════╪══════════════════════╡
│ 0                  ┆ 0.840092             │
│ 1                  ┆ 0.927329             │
└────────────────────┴──────────────────────┘


Видим, что после правильного ответа учащиеся чаще (92.7%) брали разъяснения, чем после неправильного (84%) 

Посмотрим зависимость правильности ответов от общего времени

In [66]:
result = ((train_test.filter(
    # отрезаем выбросы выше 90-го перцентиля
    (pl.col('timestamp') < pl.col('timestamp').quantile(0.9)) &
    # отрезаем выбросы ниже относительно малого значения
    (pl.col('timestamp') > 300000000)
).group_by(
    # группируем данные по времени, сгладив результаты
    (pl.col('timestamp') // 250000000).alias('time'),
).agg(
    # для аггрегации возьмем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean_correctly'),
# отсортируем по времени
)).sort('time')
          .collect(streaming=True))

# выведем линейный график
px.line(result, x='time', y='mean_correctly', )

Не заметно какого то четкого тренда, но есть сильное снижение правильных ответов при долгом большом общем времени

Для большей информативности посмотрим, как зависит среднее количество правильных ответов от полного времени обучения.

In [62]:
# группируем данные по пользователям
result = (train.group_by(
    pl.col('user_id')
# посчитаем необходимые аггрегаты
).agg(
    # количество строк
    pl.col('row_id').count(),
    # макимальное значение времени
    pl.col('timestamp').max().alias('time'),
    # среднее количество правильных ответов
    pl.col('answered_correctly').mean(),
).filter(
    # уберем из выборки результаты с малым количеством строк
    pl.col('row_id') > pl.col('row_id').quantile(0.1)
).group_by(
    # группируем по сглаженному времени
    pl.col('time') // 1000000000
).agg(
    # аггрегируем среднее по правильным ответам
    pl.col('answered_correctly').mean()
).sort('time')
          .collect(streaming=True))

# строим график
px.line(result, x='time', y='answered_correctly')

На графике хороша видна прямая зависимость количества правильных ответов от общего времени.

Посмотрим на зависимость правильных ответов от затраченного на вопрос времени

In [63]:
result = ((train.filter(
    # отбросим слишком большие значения времени
    (pl.col('prior_question_elapsed_time') < pl.col('prior_question_elapsed_time').quantile(0.95)) &
    # отбросим слишком малые значения времени
    (pl.col('prior_question_elapsed_time') > pl.col('prior_question_elapsed_time').quantile(0.05))
).select(
    # для соответствия столбцов делаем сдвиг
    pl.col('prior_question_elapsed_time').slice(1, None),
    pl.col('answered_correctly').slice(0, pl.col('answered_correctly').count() - 1)
).group_by(
    # группируем по сглаженному до секунд времени
    (pl.col('prior_question_elapsed_time') // 1000).alias('time')
# считаем среднее количество правильных ответов
).agg(pl.col('answered_correctly').mean()))
          .sort(pl.col('time'))
          .collect(streaming=True))

# строим график
px.line(result, x='time', y='answered_correctly')

Не видим сильной зависимости от времени ответа, при жирм наблюдается 2 ярко выраженных пика на 43 секундах и 58 секундах.

Посмотрим на связь с метаданными по вопросам.
Сначала подготовим объекты выборки

In [68]:
# джойним датафрэймы с результатами и метаинформацией
quest_joined_q = (train_test.join(questions_data, left_on='content_id', right_on='question_id', how='left'))
# в столбце с тэгами данные преобразовываем в список
quest_joined_split_q = quest_joined_q.with_columns(
    pl.col('tags').str.split(" ")
)

Найдем темы с наибольшим и наименьшим средним количеством правильных ответов.

In [69]:
# берем из датафрэйма столбцы с тэгами и правильностью ответов
quest_joined_expl_q = (quest_joined_split_q.select(
    pl.col('answered_correctly'),
    pl.col('tags'),
# разбиваем столбец с тэгами на отдельные строки
).explode('tags')
# группируем по тэгу
.group_by(
    pl.col('tags').alias('tag')
).agg(
    # считаем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean'),
).drop_nulls()
# и сортируем по нему
.sort('mean'))

print(quest_joined_expl_q.collect(streaming=True))


shape: (188, 2)
┌─────┬──────────┐
│ tag ┆ mean     │
│ --- ┆ ---      │
│ str ┆ f64      │
╞═════╪══════════╡
│ 24  ┆ 0.421025 │
│ 19  ┆ 0.438666 │
│ 23  ┆ 0.467508 │
│ 50  ┆ 0.50423  │
│ 11  ┆ 0.520857 │
│ …   ┆ …        │
│ 40  ┆ 0.850398 │
│ 149 ┆ 0.874416 │
│ 130 ┆ 0.874963 │
│ 187 ┆ 0.875299 │
│ 68  ┆ 0.89838  │
└─────┴──────────┘


Видим, что на вопросы с тэгом=24 получено всего 42.1% правильных ответов, а на вопросы с тэгом=68 более 89.8%

Теперь посмотрим на такую же статистику по разделам.

In [70]:
# берем из датафрэйма столбцы с темами и правильностью ответов
quest_joined_expl_q = (quest_joined_split_q.select(
    pl.col('answered_correctly'),
    pl.col('part'),
# группируем по разделу
).group_by(
    pl.col('part')
).agg(
    # считаем среднее количество правильных ответов
    pl.col('answered_correctly').mean().alias('mean'),
).drop_nulls()
                       # и сортируем по нему
                       .sort('mean'))

print(quest_joined_expl_q.collect(streaming=True))

shape: (7, 2)
┌──────┬──────────┐
│ part ┆ mean     │
│ ---  ┆ ---      │
│ i64  ┆ f64      │
╞══════╪══════════╡
│ 4    ┆ 0.638425 │
│ 5    ┆ 0.659624 │
│ 7    ┆ 0.660839 │
│ 6    ┆ 0.679117 │
│ 3    ┆ 0.71614  │
│ 2    ┆ 0.769356 │
│ 1    ┆ 0.782284 │
└──────┴──────────┘


Видим, что на вопросы с темой=5 получено 63.8% правильных ответов, а на вопросы с темой=1 более 78.2%


In [39]:
# группируем данные по пользователям
result = (train.group_by('user_id')
     # посчитаем необходимые аггрегаты
.agg(
    # количество строк
    pl.col('row_id').count().alias('rows'),
    # количество лекций
    (pl.col('content_type_id') == 1).sum().alias('lectures'),
    # количество вопросов
    (pl.col('content_type_id') == 0).sum().alias('questions'),
    # количество правильных ответов
    (pl.col('answered_correctly') == 1).sum().alias('correct'),
).filter(
    # уберем из выборки результаты с малым количеством строк
    (pl.col('rows') > pl.col('rows').quantile(0.1)) &
    # Отсечем выбросы с количеством лекций
    (pl.col('lectures') < pl.col('lectures').quantile(0.95))
# отбираем колонки для дальнейшего анализа
).select(
    # количество лекций
    pl.col('lectures'),
    # считаем процент правильных ответов
    (pl.col('correct') / pl.col('questions') * 100).alias('correct_%')
).group_by(
    # группируем по количеству лекций
    pl.col('lectures')
).agg(
    # аггрегируем среднее по правильным ответам
    pl.col('correct_%').mean()
).sort('lectures')
          .collect(streaming=True))
# строим график
print(result)
px.line(result, x='lectures', y='correct_%')

shape: (27, 2)
┌──────────┬───────────┐
│ lectures ┆ correct_% │
│ ---      ┆ ---       │
│ u32      ┆ f64       │
╞══════════╪═══════════╡
│ 0        ┆ 51.087413 │
│ 1        ┆ 59.436572 │
│ 2        ┆ 62.151852 │
│ 3        ┆ 64.122743 │
│ 4        ┆ 65.164636 │
│ …        ┆ …         │
│ 22       ┆ 69.97873  │
│ 23       ┆ 69.712456 │
│ 24       ┆ 69.760721 │
│ 25       ┆ 70.56992  │
│ 26       ┆ 70.248633 │
└──────────┴───────────┘


На графике хорошо видна прямая зависимость оценки от количества посещенных лекций, при этом после 10 лекций процент успешных оценок растёт не существенно. Стремительный же рост виден уже после первой лекции.